In [1]:
# import the necessary packages
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, ELU, Dropout
from sklearn.model_selection import train_test_split
from keras.backend import tf as ktf
from keras.optimizers import Adam
from scipy.misc import imresize
from keras.callbacks import ModelCheckpoint
import csv
import matplotlib.image as mpimg
import cv2
import matplotlib.pyplot as plt
import random
from random import randint

In [2]:
#read csv generated from the driving simulator
lines = []
with open('./data/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [3]:
def change_path_from_to(path, from_path="IMG",to_path="./data/data/IMG"):
    return path.replace(from_path, to_path)

In [4]:
import numpy as np
def flip_image_and_steering(img, steering):
    return (np.fliplr(img), -steering)

In [5]:
def resize_img(img): 
    return imresize(img[35:135, :], (66, 208, 3), interp='bilinear', mode=None)

In [6]:
#Thanks for Vivek Yadav and his work here, https://chatbotslife.com/using-augmentation-to-mimic-human-driving-496b569760a9
# I was able to augment the data with more images using the following methods
    
#random brightness    
def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    image1 = np.array(image1, dtype = np.float64)
    random_bright = .5+np.random.uniform()
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1[:,:,2][image1[:,:,2]>255]  = 255
    image1 = np.array(image1, dtype = np.uint8)
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

#random shiftts
def trans_image(image,steer,trans_range):
    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    steer_ang = steer + tr_x/trans_range*2*.2
    tr_y = 40*np.random.uniform()-40/2
    #tr_y = 0
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
    image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))
    
    return image_tr,steer_ang, tr_x

### Let us test the augment_brightness_camera_images on one of the images

In [7]:
image = mpimg.imread(change_path_from_to(lines[132][0].strip()))
rows,cols,channels = image.shape
steer = float(lines[132][3])

In [8]:
plt.imshow(image)
plt.show()
plt.imshow(augment_brightness_camera_images(image))
plt.show()

In [9]:
image_tr,steer_ang,tr_x = trans_image(image,steer,80)
plt.imshow(image_tr)
plt.show()

### Let us look at some samples after we apply the random shifting for one of the image

In [10]:
plt.figure(figsize=(12,10))
plt.subplot(4,3,1)
plt.imshow(image)
plt.title('str: ' + str(np.round(steer,2)))
plt.axis('off')

for i in range(11):
    plt.subplot(4,3,i+2)
    image_tr,steer_ang,tr_x = trans_image(image,steer,50)
    plt.title('str: ' + str(np.round(steer_ang,2)) )
    plt.imshow(image_tr)
    plt.axis('off')
plt.show()

In [11]:
def load_image(imagepath):    
    im = mpimg.imread(imagepath, 1)
    return imresize(im[45:135, :], (66, 208, 3), interp='bilinear', mode=None)

In [12]:
#our X
car_images = []

#our Y - steering angle 
steering_angles = []

In [13]:
i = 0
flag = True

for line in lines:
    if i == 0:
        i = i + 1
        continue
    
    #get corresponding streeing angle to Y
    steering_center = float(line[3])       
    
    # read in images from center, left and right cameras
    img_center = load_image(change_path_from_to(line[0].strip()))
    img_left = load_image(change_path_from_to(line[1].strip()))
    img_right = load_image(change_path_from_to(line[2].strip()))    
    
#     # read in images from center, left and right cameras
#     img_center = cv2.imread(change_path_from_to(line[0].strip()))
#     img_left = cv2.imread(change_path_from_to(line[1].strip()))
#     img_right = cv2.imread(change_path_from_to(line[2].strip()))

   
    # create adjusted steering measurements for the side camera images
    correction = 0.20 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction    
    
   
    #lets start augmenting data for these set of images

    #flip the center , left and right images and add to the augmented list 
    img_center_flipped, steering_center_flipped = flip_image_and_steering(img_center, steering_center)
    img_left_flipped, steering_left_flipped = flip_image_and_steering(img_left, steering_left)
    img_right_flipped, steering_right_flipped = flip_image_and_steering(img_right, steering_right)
    
    
    
    aug_list_of_images = [img_center, img_left, img_right, img_center_flipped, img_left_flipped, img_right_flipped, 
                          augment_brightness_camera_images(img_center)]
    
    aug_list_of_steering_angles = [steering_center, steering_left, steering_right, 
                                   steering_center_flipped, steering_left_flipped, steering_right_flipped, steering_center]    
    
#     #lets augment some more data by using brightness adjustment and random shifting 
#     indx = randint(1,3)
#     if indx == 1:
#         img_to_change_and_append = img_center
#         flipped_img_to_change_and_append = img_center_flipped
#         steering_to_append = steering_center
#     elif indx == 2:
#         img_to_change_and_append = img_left
#         flipped_img_to_change_and_append = img_left_flipped
#         steering_to_append = steering_left
#     else:
#         img_to_change_and_append = img_right
#         flipped_img_to_change_and_append = img_right_flipped
#         steering_to_append = steering_right
    

#     #for the image, get a copy with some adjusted brightness
#     aug_list_of_images.append(augment_brightness_camera_images(img_to_change_and_append))
#     aug_list_of_steering_angles.append(steering_to_append)
    
#     #randomly shift the image and get the corresponding sheered image and adjusted steering angle 
#     image_tr,steer_ang,tr_x = trans_image(img_to_change_and_append,steering_to_append,80)
#     aug_list_of_images.append(image_tr)
#     aug_list_of_steering_angles.append(steer)
    
    
#     #adjust the brightness for flipped image and add it to augmented list
#     aug_list_of_images.append(augment_brightness_camera_images(flipped_img_to_change_and_append))
#     aug_list_of_steering_angles.append(-steering_to_append)


    #for the image, get a copy with some adjusted brightness
#     aug_list_of_images.append(augment_brightness_camera_images(img_center))
#     aug_list_of_steering_angles.append(steering_center)
    
#     aug_list_of_images.append(augment_brightness_camera_images(img_center))
#     aug_list_of_steering_angles.append(steering_center)    
    
#     if flag:
#         #for the image, get a copy with some adjusted brightness
#         aug_list_of_images.append(augment_brightness_camera_images(img_left))
#         aug_list_of_steering_angles.append(steering_left)
#         flag = False
#     else:
#         #for the image, get a copy with some adjusted brightness
#         aug_list_of_images.append(augment_brightness_camera_images(img_right))
#         aug_list_of_steering_angles.append(steering_right)
#         flag = True





#     aug_list_of_images.append(augment_brightness_camera_images(img_center))
#     aug_list_of_steering_angles.append(steering_center) 



#     #randomly shift the image and get the corresponding sheered image and adjusted steering angle 
#     if flag:
#         image_tr,steer_ang,tr_x = trans_image(img_center,steering_center,50)
#         aug_list_of_images.append(image_tr)
#         aug_list_of_steering_angles.append(steer)
#         flag = False
#     else:
#         aug_list_of_images.append(augment_brightness_camera_images(img_center))
#         aug_list_of_steering_angles.append(steering_center)    
#         flag = True
    
#     #adjust the brightness for flipped image and add it to augmented list
#     aug_list_of_images.append(augment_brightness_camera_images(img_center_flipped))
#     aug_list_of_steering_angles.append(steering_center_flipped)



    #finally add to the final list 
    car_images.extend(aug_list_of_images)
    steering_angles.extend(aug_list_of_steering_angles)

In [ ]:
len(car_images)

In [ ]:
len(steering_angles)

In [14]:
# i = 0
# flag = True

# for line in lines:
#     if i == 0:
#         i = i + 1
#         continue
    
    
#     probability = random.random()
    
#     #get corresponding streeing angle to Y
#     steering_center = float(line[3])
    
#     #since majority of the data is biased towards steering at 0 angle, we need nullify the effect of this bias
#     #we half of the straight steering images out so that we have a more balanced dataset
# #     if (probability < 0.35 and abs(steering_center) < 0.001):
# #         continue
        
        
    
#     # read in images from center, left and right cameras
#     img_center = cv2.imread(change_path_from_to(line[0].strip()))
#     img_left = cv2.imread(change_path_from_to(line[1].strip()))
#     img_right = cv2.imread(change_path_from_to(line[2].strip()))


    
#     # create adjusted steering measurements for the side camera images
#     correction = 0.25 # this is a parameter to tune
#     steering_left = steering_center + correction
#     steering_right = steering_center - correction    
    
#     aug_list_of_images = [img_center, img_left, img_right]
#     aug_list_of_steering_angles = [steering_center, steering_left, steering_right]
    
#     #lets start augmenting data for these set of images

#     #flip the center , left and right images and add to the augmented list 
#     img_center_flipped, steering_center_flipped = flip_image_and_steering(img_center, steering_center)
#     img_left_flipped, steering_left_flipped = flip_image_and_steering(img_left, steering_left)
#     img_right_flipped, steering_right_flipped = flip_image_and_steering(img_right, steering_right)
    
#     aug_list_of_images.append(img_center_flipped)
#     aug_list_of_steering_angles.append(steering_center_flipped)

#     aug_list_of_images.append(img_left_flipped)
#     aug_list_of_steering_angles.append(steering_left_flipped)
    
#     aug_list_of_images.append(img_right_flipped)
#     aug_list_of_steering_angles.append(steering_right_flipped)
    
    
# #     #lets augment some more data by using brightness adjustment and random shifting 
# #     indx = randint(1,3)
# #     if indx == 1:
# #         img_to_change_and_append = img_center
# #         flipped_img_to_change_and_append = img_center_flipped
# #         steering_to_append = steering_center
# #     elif indx == 2:
# #         img_to_change_and_append = img_left
# #         flipped_img_to_change_and_append = img_left_flipped
# #         steering_to_append = steering_left
# #     else:
# #         img_to_change_and_append = img_right
# #         flipped_img_to_change_and_append = img_right_flipped
# #         steering_to_append = steering_right
    

# #     #for the image, get a copy with some adjusted brightness
# #     aug_list_of_images.append(augment_brightness_camera_images(img_to_change_and_append))
# #     aug_list_of_steering_angles.append(steering_to_append)
    
# #     #randomly shift the image and get the corresponding sheered image and adjusted steering angle 
# #     image_tr,steer_ang,tr_x = trans_image(img_to_change_and_append,steering_to_append,80)
# #     aug_list_of_images.append(image_tr)
# #     aug_list_of_steering_angles.append(steer)
    
    
# #     #adjust the brightness for flipped image and add it to augmented list
# #     aug_list_of_images.append(augment_brightness_camera_images(flipped_img_to_change_and_append))
# #     aug_list_of_steering_angles.append(-steering_to_append)


#     #for the image, get a copy with some adjusted brightness
# #     aug_list_of_images.append(augment_brightness_camera_images(img_center))
# #     aug_list_of_steering_angles.append(steering_center)
    
# #     aug_list_of_images.append(augment_brightness_camera_images(img_center))
# #     aug_list_of_steering_angles.append(steering_center)    
    
# #     if flag:
# #         #for the image, get a copy with some adjusted brightness
# #         aug_list_of_images.append(augment_brightness_camera_images(img_left))
# #         aug_list_of_steering_angles.append(steering_left)
# #         flag = False
# #     else:
# #         #for the image, get a copy with some adjusted brightness
# #         aug_list_of_images.append(augment_brightness_camera_images(img_right))
# #         aug_list_of_steering_angles.append(steering_right)
# #         flag = True
    
# #     #randomly shift the image and get the corresponding sheered image and adjusted steering angle 
# #     image_tr,steer_ang,tr_x = trans_image(img_center,steering_center,80)
# #     aug_list_of_images.append(image_tr)
# #     aug_list_of_steering_angles.append(steer)
    
    
# #     #adjust the brightness for flipped image and add it to augmented list
# #     aug_list_of_images.append(augment_brightness_camera_images(img_center_flipped))
# #     aug_list_of_steering_angles.append(steering_center_flipped)



#     #finally add to the final list 
#     car_images.extend(aug_list_of_images)
#     steering_angles.extend(aug_list_of_steering_angles)

In [15]:
#crop images to match the 66x208 nvidia architecture input
#keras needs us to convert to numpy arrays
# X_train = np.array(list(map(resize_img, car_images)))
# train_samples, validation_samples, train_labels, validation_labels = train_test_split(car_images, steering_angles, test_size=0.2, random_state=42)

# resized_car_images = []
# for i in car_images:
#     resized_car_images.append(resize_img(i))

X_train = np.array(car_images)
y_train = np.array(steering_angles)

del car_images
# del resized_car_images
del steering_angles

In [16]:
# len(train_samples)

In [17]:
# len(validation_samples)

In [18]:
# import sklearn
# from sklearn.utils import shuffle
# def generator(samples, labels, batch_size=5000):
#     num_samples = len(samples)
#     while 1: # Loop forever so the generator never terminates
#         X, y = shuffle(samples, labels)
#         for offset in range(0, num_samples, batch_size):
#             batch_samples = X[offset:offset+batch_size]
#             batch_labels = y[offset:offset+batch_size]

#             images = []
#             angles = []
          
#             for batch_sample, batch_label in zip(batch_samples, batch_labels):
#                 images.append(resize_img(batch_sample))
#                 angles.append(batch_label)

#             # trim image to only see section with road
#             X_train = np.array(images)
#             y_train = np.array(angles)
#             yield sklearn.utils.shuffle(X_train, y_train)

# # compile and train the model using the generator function
# train_generator = generator(train_samples, train_labels, batch_size=32)
# validation_generator = generator(validation_samples, validation_labels, batch_size=32)

### Augmenting more data with sheering

In [19]:
class nvidia_model:
    #http://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/
    @staticmethod
    def build(width=208, height=66, depth=3):
        """
        width: The width of our input images.
        height: The height of our input images.
        depth: The depth (i.e., number of channels) of our input images.
        """
        
        # initialize the model
        model = Sequential()
        #preprocessing step - 1
        # divide by max value i.e. 255 to bring values between 0 and 1
        #subtract 0.5 to center around 0        
        model.add(Lambda(lambda x:x / 255 - 0.5, input_shape=(height, width, depth)))
                          
        #add 3 layers of conv2d , (output depth 24, 36, and 48), each with 2x2 stride
        model.add(Conv2D(24, (5, 5), activation="elu",  kernel_initializer="he_normal", strides=(2,2), padding='valid'))
        model.add(Conv2D(36, (5, 5), activation="elu",  kernel_initializer="he_normal", strides=(2,2), padding='valid'))
        model.add(Conv2D(48, (5, 5), activation="elu",  kernel_initializer="he_normal", strides=(2,2), padding='valid'))
        model.add(Conv2D(64, (3, 3), activation="elu",  kernel_initializer="he_normal", strides=(1,1), padding='valid'))
        model.add(Conv2D(64, (3, 3), activation="elu",  kernel_initializer="he_normal", strides=(1,1), padding='valid'))
        model.add(Flatten())
        model.add(Dense(100, kernel_initializer='he_normal'))
        model.add(ELU())
        model.add(Dense(50, kernel_initializer='he_normal'))
        model.add(ELU())
        model.add(Dense(10, kernel_initializer='he_normal'))
        model.add(ELU())
        model.add(Dense(1, kernel_initializer='he_normal'))
                  
        # return the constructed network architecture
        return model        

In [20]:
model = nvidia_model.build()


# checkpoint
checkpoint = ModelCheckpoint("model-{epoch:02d}.h5", monitor='loss', verbose=1, save_best_only=False, mode='max')

# model.compile(loss='mse', optimizer='adam')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss='mse')    
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7, callbacks=[checkpoint])

model.save('model_nvidia_v3.h5')


Train on 45001 samples, validate on 11251 samples
Epoch 1/7
45001/45001 [==============================] - 500s - loss: 0.0232 - val_loss: 0.0198
Epoch 2/7
45001/45001 [==============================] - 544s - loss: 0.0154 - val_loss: 0.0209
Epoch 3/7
45001/45001 [==============================] - 567s - loss: 0.0133 - val_loss: 0.0192
Epoch 4/7
45001/45001 [==============================] - 538s - loss: 0.0117 - val_loss: 0.0199
Epoch 5/7
45001/45001 [==============================] - 509s - loss: 0.0106 - val_loss: 0.0201
Epoch 6/7
45001/45001 [==============================] - 500s - loss: 0.0095 - val_loss: 0.0199
Epoch 7/7
45001/45001 [==============================] - 502s - loss: 0.0088 - val_loss: 0.0192


In [21]:
X_train.shape

(56252, 66, 208, 3)

In [23]:
y_train.shape

(56252,)

In [ ]:
# history = model.fit_generator((train_generator), steps_per_epoch = len(train_samples), epochs = 7,
#                     verbose=1, callbacks=[checkpoint], validation_data=validation_generator, validation_steps=len(validation_samples))